In [111]:
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import os
import pickle
from sklearn.metrics import recall_score,roc_auc_score

# Loading important files:

In [95]:
e1=0
e2=0
e3=0
e4=0
#loading data
try:
    data=pd.read_csv('sensor.csv',index_col=0)
    time=list(data['timestamp'].loc[9:].values)
except:
    e1=1

#loading normalizer
try:
    if not os.path.isfile('model_scaler_s10_std'):
        with ZipFile('model_scaler.zip', 'r') as zipObj:
            zipObj.extractall()
        zipObj.close()
except:
    e2=1
    
#loading model
try:
    model = pickle.load(open('final_model', 'rb'))
except:
    e3=1
#na replacement load
try:
    na_replace = np.load('NA_replace')
except:
    e4=1

# Final function:

In [96]:
def function_1(samples):
    if e1==1:
        print('Please store sensor.csv file in current directory, it is not found please check and rerun whole notebook')
    if e2==1:
        print('Please store model_scaler in current directory, it is not found please check and rerun whole notebook')
    if e3==1:
        print('Please store final_model in current directory, it is not found please check and rerun whole notebook')
    if e4==1:
        print('Please store NA_replace in current directory, it is not found please check and rerun whole notebook')
    
    s=0
    for sample in samples:
        if len(samples.shape)==1:
            sample=samples
        if sample[0] not in time:
            return "invalid data point given please check timestamp is should be in range '2018-04-01 00:59:00' to '2018-08-31 23:59:00'"
        if len(sample) !=53:
            return 'all the 52 reading from the sensor is not given please check'
        index=data[data['timestamp']==sample[0]].index[0]
        window=data.loc[index-9:index-1].copy()
        window.drop(['machine_status'],inplace=True,axis=1)
        window=window.append(pd.Series(sample, index=window.columns), ignore_index=True)
        window.drop(['sensor_01','sensor_03','sensor_14','sensor_15','sensor_16','sensor_17','sensor_18','sensor_19','sensor_20','sensor_21',
           'sensor_22','sensor_23','sensor_24','sensor_25','sensor_26','sensor_27','sensor_28','sensor_29','sensor_30',
           'sensor_31','sensor_33','sensor_34','sensor_37','sensor_36','sensor_48','timestamp'],
            inplace=True,axis=1)
    
        for i in range(len(window.columns)):
            window[window.columns[i]]=window[window.columns[i]].fillna(na_replace[i])
    
        columns=[]
        for col in window.columns:
            columns.append('s{0}_median'.format(col[7:])) #to select sensor number
            columns.append('s{0}_mean'.format(col[7:]))
            columns.append('s{0}_std'.format(col[7:]))
            columns.append('s{0}_min'.format(col[7:]))
            columns.append('s{0}_max'.format(col[7:]))
        features=[]
        data_window=pd.DataFrame(columns=columns)
        for col in window.columns:
                features.append(window[col].median())#calculating features
                features.append(window[col].mean())
                features.append(window[col].std())
                features.append(window[col].min())
                features.append(window[col].max())
        data_window=data_window.append(pd.Series(features, index=columns), ignore_index=True)

        for col in data_window.columns:
            scaler = pickle.load(open('model_scaler_{0}'.format(col), 'rb'))
            data_window[col]=scaler.transform(data_window[col].values.reshape(1, -1))
        label=model.predict(data_window)
        if label==0:
            print('Machine status for sample {0} at time {1} : "NORMAL"'.format(s,sample[0])) 
        else:
            print('Machine status for sample {0} at time {1} : "BROKEN"'.format(s,sample[0]))
        if len(samples.shape)==1:
            break
        s=s+1

In [109]:
def function_2(samples):
    if e1==1:
        print('Please store sensor.csv file in current directory, it is not found please check and rerun whole notebook')
    if e2==1:
        print('Please store model_scaler in current directory, it is not found please check and rerun whole notebook')
    if e3==1:
        print('Please store final_model in current directory, it is not found please check and rerun whole notebook')
    if e4==1:
        print('Please store NA_replace in current directory, it is not found please check and rerun whole notebook')
    
    s=0
    pred_labels=[]
    true_labels=[]
    pred_proba=[]
    for sample in samples:
        if len(samples.shape)==1:
            sample=samples
        if sample[0] not in time:
            return "invalid data point given please check timestamp is should be in range '2018-04-01 00:59:00' to '2018-08-31 23:59:00'"
        if len(sample) !=54:
            return 'all the 52 reading from the sensor with timestamp and class label is not given please check'
        true_labels.append(sample[-1])
        index=data[data['timestamp']==sample[0]].index[0]
        window=data.loc[index-9:index-1].copy()
        window=window.append(pd.Series(sample, index=window.columns), ignore_index=True)
        window.drop(['sensor_01','sensor_03','sensor_14','sensor_15','sensor_16','sensor_17','sensor_18','sensor_19','sensor_20','sensor_21',
           'sensor_22','sensor_23','sensor_24','sensor_25','sensor_26','sensor_27','sensor_28','sensor_29','sensor_30',
           'sensor_31','sensor_33','sensor_34','sensor_37','sensor_36','sensor_48','timestamp','machine_status'],
            inplace=True,axis=1)
    
        for i in range(len(window.columns)):
            window[window.columns[i]]=window[window.columns[i]].fillna(na_replace[i])
    
        columns=[]
        for col in window.columns:
            columns.append('s{0}_median'.format(col[7:])) #to select sensor number
            columns.append('s{0}_mean'.format(col[7:]))
            columns.append('s{0}_std'.format(col[7:]))
            columns.append('s{0}_min'.format(col[7:]))
            columns.append('s{0}_max'.format(col[7:]))
        features=[]
        data_window=pd.DataFrame(columns=columns)
        for col in window.columns:
                features.append(window[col].median())#calculating features
                features.append(window[col].mean())
                features.append(window[col].std())
                features.append(window[col].min())
                features.append(window[col].max())
        data_window=data_window.append(pd.Series(features, index=columns), ignore_index=True)

        for col in data_window.columns:
            scaler = pickle.load(open('model_scaler_{0}'.format(col), 'rb'))
            data_window[col]=scaler.transform(data_window[col].values.reshape(1, -1))
        label=model.predict(data_window)
        pred_proba.append(model.predict_proba(data_window)[:,1].tolist()[0])
        pred_labels.append(label)
        if len(samples.shape)==1:
            break
        s=s+1
    print('Recall score is:',recall_score(true_labels,pred_labels))
    try:
        print('AUC score is:',roc_auc_score(true_labels,pred_proba))
    except ValueError:
        pass
    

In [105]:
def get_machine_status(start,end):
    '''
    return class label for each sample. it take each sample and check machine status after 5 minute in window of size 10,
    if there is 'BROKEN'  or 'RECOVERING' present in 10 minutes window then class label is 1.
    '''
    start_index=data[data['timestamp']==start].index[0]
    end_index=data[data['timestamp']==end].index[0]
    label=[]
    for index in range(start_index,end_index+1):
        a=data['machine_status'].loc[index+5:index+15].values
        if (('BROKEN' in a) or ('RECOVERING' in a)):
            label.append(1)
        else:
            label.append(0)
    return label
            
    

## Failure dates with recovering time:
- failure 1 at 2018-04-12 21:55:00, recovering time is 15.733333333333333 hours
- failure 2 at 2018-04-18 00:30:00, recovering time is 51.833333333333336 hours
- failure 3 at 2018-05-19 03:18:00, recovering time is 21.866666666666667 hours
- failure 4 at 2018-05-25 00:30:00, recovering time is 10.083333333333334 hours
- failure 5 at 2018-06-28 22:00:00, recovering time is 139.83333333333334 hours
- failure 6 at 2018-07-08 00:11:00, recovering time is 0.6833333333333333 hours
- failure 7 at 2018-07-25 14:00:00, recovering time is 1.25 hours

## Function 1 : predicting machine status after 5 minutes

In [87]:
function_1(data[data['timestamp']=='2018-06-28 21:40:00'][data.columns[0:-1]].values)

Machine status for sample 0 at time 2018-06-28 21:40:00 : "BROKEN"


In [88]:
function_1(data[data['timestamp']=='2018-07-08 00:06:00'][data.columns[0:-1]].values)

Machine status for sample 0 at time 2018-07-08 00:06:00 : "BROKEN"


In [89]:
function_1(data[data['timestamp']=='2018-07-08 00:00:00'][data.columns[0:-1]].values)

Machine status for sample 0 at time 2018-07-08 00:00:00 : "NORMAL"


In [90]:
function_1(data[data['timestamp']=='2018-07-25 03:00:00'][data.columns[0:-1]].values)

Machine status for sample 0 at time 2018-07-25 03:00:00 : "NORMAL"


In [91]:
function_1(data[data['timestamp']=='2018-07-25 13:55:00'][data.columns[0:-1]].values)

Machine status for sample 0 at time 2018-07-25 13:55:00 : "BROKEN"


In [92]:
function_1(data[data.columns[0:-1]].iloc[4604:4609].values)

Machine status for sample 0 at time 2018-04-04 04:44:00 : "NORMAL"
Machine status for sample 1 at time 2018-04-04 04:45:00 : "NORMAL"
Machine status for sample 2 at time 2018-04-04 04:46:00 : "NORMAL"
Machine status for sample 3 at time 2018-04-04 04:47:00 : "NORMAL"
Machine status for sample 4 at time 2018-04-04 04:48:00 : "NORMAL"


## Function 2 : printing recall score for given  dataset
- to get recall score, 1st we have to get class label for given sample after 5 minutes, hence we have written a function nameget_machine_status.


In [110]:
start='2018-04-12 21:55:00'
end='2018-04-12 22:00:00'
label=get_machine_status(start, end)
val_data=data.loc[data[data['timestamp']==start].index[0]:data[data['timestamp']==end].index[0]]
val_data['machine_status']=label
function_2(val_data.values)

D:\study\anna\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


Recall score is: 1.0


In [112]:
start='2018-04-12 21:55:00'
end='2018-04-14 00:30:00'
label=get_machine_status(start, end)
val_data=data.loc[data[data['timestamp']==start].index[0]:data[data['timestamp']==end].index[0]]
val_data['machine_status']=label
function_2(val_data.values)

D:\study\anna\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


Recall score is: 0.9936170212765958
AUC score is: 0.9998653995848469


In [113]:
start='2018-07-25 13:50:00'
end='2018-07-25 17:00:00'
label=get_machine_status(start, end)
val_data=data.loc[data[data['timestamp']==start].index[0]:data[data['timestamp']==end].index[0]]
val_data['machine_status']=label
function_2(val_data.values)

D:\study\anna\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


Recall score is: 0.9506172839506173
AUC score is: 0.941189674523008
